___
## Módulo 5: Tratamiento de series temporales.
___

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
#%matplotlib inline

### 5.1. Tratamiento de series temporales I

**5.1.1.** Genera un vector de fechas con 15 periodos de frecuencia mensual que empiece el 2015-1-1.

In [ ]:
dates = pd.date_range(start = '2015-1-1', periods=15, freq='MS')
dates

**5.1.2.** A continuación, genera 4 columnas de números aleatorios - que sigan una distribución normal estándar centrada en 10 y con una desviación típica de 2 - de la misma longitud que el vector de fechas. Redondea los números aleatorios a 6 decimales. 

In [ ]:
dates.shape

In [ ]:
cols = np.random.normal(loc=10, scale= 2, size=(dates.shape[0],4))
cols

**5.1.3.** Genera a partir de ambos inputs un DataFrame que tenga el vector de fechas como índice.

In [ ]:
data = pd.DataFrame(cols, index=dates)
data

**5.1.4.** Accede a todos los elementos del año 2015, a todos los datos de abril del mismo año, y a los datos que se encuentran entre abril y septiembre del mismo año.

In [ ]:
data.loc['2015']

In [ ]:
data.loc['2015-04']

In [ ]:
data.loc['2015-04':'2015-09']

**5.1.5.** Convierte los siguientes strings en fechas que pandas pueda entender

- 07-07-2015
- 2015, 7, 3
- 4th of July, 2015
- 20150708
- 2015-Jul-6
- 12/11/1979

In [ ]:
from datetime import datetime

In [ ]:
dates = pd.to_datetime(['07-07-2015', datetime(2015, 7, 3), '4th of July, 2015',
                       '20150708', '2015-Jul-6', '12/11/1979'])

dates

De necesitarlo, podemos indicar el formato de entrada (qué viene antes, el día, el mes, el año...)

In [ ]:
pd.to_datetime('12/11/1979', format='%d/%m/%Y')

**5.1.6.** Construye un rango de fechas que empiece el 1 de enero de 2020, tenga 10 periodos con su frecuencia sea cada 9 días laborables.

In [ ]:
pd.date_range('2020-01-01', periods=10, freq='9B')

___
### 5.2. Tratamiento de series temporales II

**5.2.1.** Carga los csvs de datos ibex_div, ibex, NTGY, REE, SAN, pon la fecha como índice.

Esta sería una manera de hacerlo

In [ ]:
san = pd.read_csv('SAN.csv', parse_dates=True, index_col=0)
ree = pd.read_csv('REE.csv', parse_dates=True, index_col=0)
ntgy = pd.read_csv('NTGY.csv', parse_dates=True, index_col=0)
ibex = pd.read_csv('ibex.csv', parse_dates=True, index_col=0)
ibex_div = pd.read_csv('ibex_div.csv', parse_dates=True, index_col=0)

Esta sería otra manera de hacerlo (ojo, los activos no tienen lo mismos días cotizados)

In [ ]:
df = pd.DataFrame(
    {'san': pd.read_csv('SAN.csv', parse_dates=True, index_col=0).close,
     'ree': pd.read_csv('REE.csv', parse_dates=True, index_col=0).close,
     'ntgy': pd.read_csv('NTGY.csv', parse_dates=True, index_col=0).close,
     'ibex': pd.read_csv('ibex.csv', parse_dates=True, index_col=0).close,
     'ibex_div': pd.read_csv('ibex_div.csv', parse_dates=True, index_col=0).close,
    }
)

df

Quitamos las filas con NAs para que todos los activos tengan la misma longitud

In [ ]:
df = df.dropna()
df

Si graficamos directamente el DF, las diferencias de escala no nos permiten ver bien el gráfico

In [ ]:
df.plot()

Podemos hacer que todas las series empiecen en el mismo punto, dividiendo cada valor de las columnas, por el valor que tienen en la 1ª fila (vemos los rendimientos)

In [ ]:
ajustado = df / df.iloc[0]
ajustado.plot()

**5.2.2.**  Calcula el retorno anualizado del Ibex con dividendos y del Ibex.

Calculamos el número de años de la serie y el retorno total del Ibex

In [ ]:
years = (ibex.index[-1]-ibex.index[0]).days / 365
print(years)

total_return = ibex.close[-1] / ibex.close[0] - 1
total_return

Calculamos el retorno anual del Ibex mediante capitalización compuesta

In [ ]:
year_return_ibex = (1+total_return)**(1/years) - 1
year_return_ibex

Hacemos lo mismo para el Ibex con dividendos

In [ ]:
years = (ibex_div.index[-1]-ibex_div.index[0]).days / 365
print(years)

total_return = ibex_div.close[-1] / ibex_div.close[0] - 1 
total_return

In [ ]:
return_ibex_div = (1 + total_return)**(1/years) - 1
return_ibex_div

Graficamos ambas series

In [ ]:
ibex_div.close.plot()
ibex.close.plot()

Si queremos hacer lo mismo, pero usando el DF ya homogeneizado y limpito:

In [ ]:
years = (df.index[-1]-df.index[0]).days / 365
print(years)

total_return = df.ibex[-1] / df.ibex[0] - 1 
return_ibex = (1 + total_return)**(1/years) - 1
print(return_ibex)

total_return = df.ibex_div[-1] / df.ibex_div[0] - 1 
return_ibex_div = (1 + total_return)**(1/years) - 1
print(return_ibex_div)

df.ibex.plot()
df.ibex_div.plot()

**5.2.3.**  Calcula la serie de retornos anuales  del Ibex con dividendos y del Ibex.

Sacamos el primer día de cada año con resample

In [ ]:
first_day_year_value_ibex = ibex.close.resample('YS').first()
first_day_year_value_ibex_div = ibex_div.close.resample('YS').first()

first_day_year_value_ibex

Calculamos los retornos anuales

In [ ]:
returns_year_ibex = first_day_year_value_ibex.pct_change().dropna()
returns_year_ibex_div = first_day_year_value_ibex_div.pct_change().dropna()

returns_year_ibex

Podemos hacer lo mismo con el DF limpito

In [ ]:
first_day_year = df.resample('YS').first()
returns_year = first_day_year.pct_change().dropna()
returns_year

**5.2.4.**  Realiza un gráfico de barras comparándolos.

Simplificamos la fecha para que al graficar solo aparezca el año y no toda la fecha a la hora de graficar

In [ ]:
returns_year_ibex.index = returns_year_ibex.index.year 
returns_year_ibex_div.index = returns_year_ibex_div.index.year 

In [ ]:
data = pd.DataFrame({
    'ibex': returns_year_ibex,
    'ibex div': returns_year_ibex_div
})

data.plot.bar()

Podemos aplicarlo a todo el DF

In [ ]:
returns_year.index = returns_year.index.year 
returns_year.plot.bar(figsize=(20,10))

**5.2.5.**  Compara los retornos anuales del Ibex con los de SAN.

In [ ]:
df_ibex_san = pd.DataFrame({
    'san': returns_year.san,
    'ibex': returns_year.ibex,
})
df_ibex_san.plot.bar()

**5.2.6.**  Calcula la correlación del SAN, REE y NTGY con el IBEX con dividendos.

Primero calculamos los retornos logarítmicos

In [ ]:
returns_data = np.log(df).diff().dropna()

Y luego la correlación

In [ ]:
returns_data.corr()

**5.2.7.** Ahora calcula la correlación rolada de 100 días para los activos del ejercico anterior.

In [ ]:
rolling_corr = returns_data.rolling(100).corr(returns_data.ibex_div)
rolling_corr

In [ ]:
rolling_corr.plot(figsize=(20,10))

**5.2.8.** Calcula la media movil de 30 y 200 dias de REE y realiza una figura junto con la serie de precios originales

In [ ]:
SMA_30 = df.ree.rolling(30).mean()
SMA_200 = df.ree.rolling(200).mean()

df.ree.plot(figsize=(20,10))
SMA_30.plot()
SMA_200.plot()

**5.2.9.**  Usando el precio de cierre del Ibex, calcula las velas mensuales y anuales.

In [ ]:
ibex_month_candle = df.ibex.resample('MS').ohlc()
ibex_month_candle.head()

In [ ]:
ibex_year_candle = df.ibex.resample('YS').ohlc()
ibex_year_candle.head()

**5.2.10.** Píntalas utilizando un gráfico de barras.

Esta sería una manera de hacerlo, con la función que vimos en el notebook de visualización

In [ ]:
from mpl_finance import candlestick_ohlc
import matplotlib.dates as mdates

def plot_candle(df, width=.5, figsize=(10, 7), tick_formater='%Y-%m'):        
    m_data = df[['open', 'high', 'low', 'close']].values
    days_m_dates_format = mdates.date2num(df.index.values) 
    data_plot = np.column_stack((days_m_dates_format, m_data))
        
    fig, ax = plt.subplots(figsize=figsize)
    _ = candlestick_ohlc(ax,
                         data_plot,
                         width=width,
                         colorup='green',
                         colordown='red')
    _ = ax.xaxis.set_major_formatter(mdates.DateFormatter(tick_formater))
    return fig, ax

In [ ]:
plot_candle(ibex_month_candle, width=10)

In [ ]:
plot_candle(ibex_year_candle, width=100)

Esta sería otra manera de hacerlo, usando directamente mplfinance

In [ ]:
import mplfinance as fplt

In [ ]:
fplt.plot(ibex_month_candle, type = 'candle', style = 'charles') # Style es para que nos las pinte en verde y en rojo

In [ ]:
fplt.plot(ibex_year_candle, type = 'candle', style = 'charles')

**5.2.11.**  Calcula la beta de los 3 activos NTGY, REE y SAN, Recuerda:

$\beta = \frac{cov(R_m, R_s)}{var(R_m)}$

Donde $R_m$ y $R_s$ son la serie de retornos del índice y de la acción.

In [ ]:
returns_data

Calculamos la covarianza

In [ ]:
cov_mat = returns_data.cov()

In [ ]:
cov_mat

In [ ]:
cov_mat / returns_data.ibex_div.var()

De la manera anterior sacamos la matriz entera, cuando quiero únicamente la última columna. 

Adapto el código

In [ ]:
cov_mat.loc[:,['ibex_div']] / returns_data.ibex_div.var()

**5.2.12.** Ahora calcula la beta rolada de 100 días para los activos del ejercico **5.2.11.**

Calculamos la covarianza rolada de los retornos de todos los activos, con respecto a los retornos del Ibex

Calculamos la varianza rolada del ibex

Calculamos la beta

In [ ]:
rolling_cov = returns_data.rolling(100).cov(returns_data.loc[:,'ibex_div'])
rolling_var = returns_data.loc[:,'ibex_div'].rolling(100).var() 

rolling_beta = rolling_cov / rolling_var

In [ ]:
rolling_beta

Si intentamos hacer la operación rolling_cov / rolling_var python no consigue hacerla bien (como podemos ver en la tabla de arriba)

Por lo que lo hacemos paso a paso

In [ ]:
beta_san = rolling_cov.san / rolling_var
beta_ree = rolling_cov.ree / rolling_var
beta_ntgy = rolling_cov.ntgy / rolling_var

In [ ]:
beta_san.plot(figsize=(20,10))
beta_ree.plot()
beta_ntgy.plot()